# Control Plane: Foundry Agent Monitoring with the Azure AI Foundry SDK

----
This notebook focuses on the **Control Plane** perspective in Azure AI Foundry and shows how to do lightweight checks with the **Foundry SDK (2.0.0b2 preview)**.

You will learn how to:

- Inspect deployed agents (Fleet-style view)
- Review Assets (Connections, Agents)
- Understand where Quota / Rate limiting is managed (mostly in Portal)

## Table of contents

- [Control Plane overview](#control-plane-overview)
- [Portal vs SDK](#portal-vs-sdk)
- [Setup](#setup)
- [Case 1: Fleet overview](#case-1-fleet-overview)
- [Case 2: Assets](#case-2-assets)
- [Case 3: Quota](#case-3-quota)
- [Additional resources](#additional-resources)
- [Wrap-up](#wrap-up)

## Control Plane overview

### What is the Control Plane?

The Control Plane is the centralized operational surface in Azure AI Foundry for monitoring and governing AI resources across projects.

```
Control Plane = monitoring + management + security + governance
```

### Key areas



## Portal vs SDK
**Recommended usage:**
- Portal: real-time monitoring, compliance/security checks, quota management
- SDK: automation, inventory checks, CI/CD validations

| Feature | Portal | SDK (Python) |
|-----|--------|-------------|
| Fleet overview | ✅ dashboards | ⚠️ limited listing |
| Assets view | ✅ visual | ✅ programmatic |
| Compliance dashboards | ✅ real-time | ❌ not supported |
| Quota management | ✅ manage in Portal | ❌ often read-only/limited |
| Admin / RBAC | ✅ manage access | ❌ not supported |

## MS Foundry Control Plane Accessibility Across Agent Platforms

| Feature        | Foundry Agent | MS Agent Framework | Hosted Agent |
|---------------|---------------|--------------------|--------------|
| Fleet overview | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |
| Asset view    | ✅ Yes | ⚠️ with Agent Wrapper  | ✅ Yes |

**MAF onboard to MS Foundry Control Plane:**
- MAF + Foundry Agent Wrapper (AzureAIClient)
- MAF → LangGraph → Container → Hosted Agent

**Hosted Agent limitation (2026-01-07):**
- **Region availability:** Hosted Agents are currently available only in **North Central US**.
- **Cross-region deployment:** If you try to deploy Hosted Agents in another region, you may see: *“Hosted Agents are not enabled in this region”* (**HTTP 400 BadRequest**).
- **Network isolation:** Hosted Agents currently can’t be created using the standard setup in **network-isolated** Foundry resources.
- **ACR requirement:** Hosted Agents can be deployed only when **Azure Container Registry (ACR)** is in the **same resource group** and the **same region (North Central US)**.
- **Last verified:** 2026-01-07

## MS Foundry Tools Integration Support Across Agent Platforms 
| Tool | Foundry Agent | MS Agent Framework | Hosted Agent |
|-----|--------|-------------|-------------|
| bing grounding | ✅ Yes | ✅ Yes | ✅ Yes |
| code interpreter | ✅ Yes | ✅ Yes | ✅ Yes |
| browser automation | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| computer use | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| fabric (Microsoft Fabric) | ✅ Yes (Preview) | ✅ Yes | ❌ No |
| file search | ✅ Yes | ✅ Yes (RAG / AI Search) | ❌ No (Not Built-in) |
| custom function | ✅ Yes | ✅ Yes | ✅ Yes |
| image generation | ✅ Yes (Preview) | ✅ Yes (Preview) | ❌ No |
| mcp (Model Context Protocol) | ✅ Yes | ✅ Yes | ✅ Yes |




## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [ ]:
# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
import json
import os
import subprocess
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto, ConnectionType
from azure.identity import DefaultAzureCredential

load_dotenv(override=True)

possible_paths = [
    '/opt/homebrew/bin',  # macOS (Apple Silicon)
    '/usr/local/bin',     # macOS (Intel) / Linux
    '/usr/bin',           # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

In [ ]:
# Load Foundry project settings from .foundry_config.json

import os

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
TENANT_ID = config.get('TENANT_ID')
PROJECT_NAME = config.get('PROJECT_NAME', 'proj-default')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')
os.environ['_ENDPOINT'] = AZURE_AI_PROJECT_ENDPOINT or ''
os.environ['PROJECT_ENDPOINT'] = AZURE_AI_PROJECT_ENDPOINT or ''

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Project endpoint: {AZURE_AI_PROJECT_ENDPOINT}")

# Fleet overview - list deployed agents
credential = DefaultAzureCredential()
client = AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential)

# Case 1: Agent overview

From agent perspective, list **deployed agents** in the project. (The SDK does not expose every Portal dashboard field, but it is useful for automated checks.)

### Register a Foundry Agent

In [ ]:
# Shared setup: Project endpoint / model deployment name
from datetime import datetime

AZURE_AI_MODEL_DEPLOYMENT_NAME = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME")

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    print("WARNING: AZURE_AI_MODEL_DEPLOYMENT_NAME is empty.")
    print("- In Foundry Portal > Models + endpoints, check the deployment Name")
    print("  then set: export AZURE_AI_MODEL_DEPLOYMENT_NAME='<deployment-name>'")
else:
    print(f"AZURE_AI_MODEL_DEPLOYMENT_NAME = {AZURE_AI_MODEL_DEPLOYMENT_NAME}")

In [ ]:
# uuid with YYYYMMDDhhmmss
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")
print(f"uuid_suffix = {uuid_suffix}")

In [ ]:
# Method 1) Create openai_client + generate a simple response (Prompt Agent + Conversations/Responses)

agent_name = f"tracing-agent-{uuid_suffix}"
instructions = "You are a helpful assistant that answers general questions."

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    raise ValueError("AZURE_AI_MODEL_DEPLOYMENT_NAME is empty. Set it in the previous cell.")

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
 ):
    agent = None
    conversation = None
    try:
        agent = project_client.agents.create_version(
            agent_name=agent_name,
            definition=PromptAgentDefinition(
                model=AZURE_AI_MODEL_DEPLOYMENT_NAME,
                instructions=instructions,
            ),
        )
        
        print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")
    
        conversation = openai_client.conversations.create(
            items=[
                {
                    "type": "message",
                    "role": "user",
                    "content": "What is the size of South Korea in square miles?",
                }
            ],
        )
        print(f"Created conversation (id: {conversation.id})")
        
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            input="",
        )
        print(f"Response output: {response.output_text}")


    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")
        

        


# Case 2: Agent overview with Tracing

From agent perspective, list **deployed agents** in the project. (The SDK does not expose every Portal dashboard field, but it is useful for automated checks.)

## Create an App Insights resource to enable tracing

In [ ]:
# Create and connect Application Insights for tracing
from azure.mgmt.applicationinsights import ApplicationInsightsManagementClient
from azure.mgmt.applicationinsights.models import ApplicationInsightsComponent
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
from azure.mgmt.loganalytics.models import Workspace
from azure.core.exceptions import ResourceNotFoundError

print("📊 Setting up Application Insights for tracing")
print("=" * 80)

try:
    subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID')
    if not subscription_id:
        print("⚠️ AZURE_SUBSCRIPTION_ID not found in environment")
        raise ValueError("AZURE_SUBSCRIPTION_ID is required")
    
    # Step 1: Create or get Log Analytics workspace (required for App Insights)
    log_analytics_client = LogAnalyticsManagementClient(
        credential=credential,
        subscription_id=subscription_id
    )
    
    workspace_name = f"{FOUNDRY_NAME}-workspace"
    print(f"\n🔍 Step 1: Checking for Log Analytics workspace: {workspace_name}")
    
    try:
        existing_workspace = log_analytics_client.workspaces.get(
            resource_group_name=RESOURCE_GROUP,
            workspace_name=workspace_name
        )
        print(f"✅ Log Analytics workspace already exists: {existing_workspace.name}")
        workspace_id = existing_workspace.id
    except Exception:
        print(f"\n🔨 Creating new Log Analytics workspace: {workspace_name}")
        workspace_params = Workspace(
            location=LOCATION,
            tags={
                "purpose": "ai-foundry-tracing",
                "project": PROJECT_NAME
            }
        )
        
        workspace_operation = log_analytics_client.workspaces.begin_create_or_update(
            resource_group_name=RESOURCE_GROUP,
            workspace_name=workspace_name,
            parameters=workspace_params
        )
        workspace = workspace_operation.result()
        workspace_id = workspace.id
        print(f"✅ Log Analytics workspace created: {workspace.name}")
    
    print(f"   Workspace ID: {workspace_id}")
    
    # Step 2: Create Application Insights linked to the workspace
    app_insights_client = ApplicationInsightsManagementClient(
        credential=credential,
        subscription_id=subscription_id
    )
    
    app_insights_name = f"{FOUNDRY_NAME}-appinsights"
    print(f"\n🔍 Step 2: Checking for Application Insights: {app_insights_name}")
    
    try:
        existing_app_insights = app_insights_client.components.get(
            resource_group_name=RESOURCE_GROUP,
            resource_name=app_insights_name
        )
        print(f"✅ Application Insights already exists: {existing_app_insights.name}")
        print(f"   Instrumentation Key: {existing_app_insights.instrumentation_key[:8]}...")
        print(f"   Connection String: {existing_app_insights.connection_string[:50]}...")
        app_insights_resource = existing_app_insights
        
    except Exception:
        print(f"\n🔨 Creating new Application Insights resource: {app_insights_name}")
        
        app_insights_params = ApplicationInsightsComponent(
            location=LOCATION,
            kind="web",
            application_type="web",
            workspace_resource_id=workspace_id,  # Link to Log Analytics workspace
            tags={
                "purpose": "ai-foundry-tracing",
                "project": PROJECT_NAME
            }
        )
        
        app_insights_resource = app_insights_client.components.create_or_update(
            resource_group_name=RESOURCE_GROUP,
            resource_name=app_insights_name,
            insight_properties=app_insights_params
        )
        
        print(f"✅ Application Insights created: {app_insights_resource.name}")
        print(f"   Instrumentation Key: {app_insights_resource.instrumentation_key[:8]}...")
        print(f"   Connection String: {app_insights_resource.connection_string[:50]}...")
    
    # Store connection string in environment for tracing
    connection_string = app_insights_resource.connection_string
    os.environ['APPLICATIONINSIGHTS_CONNECTION_STRING'] = connection_string
    

    
except Exception as e:
    print(f"\n⚠️ Failed to set up Application Insights: {e}")
    print("\nTroubleshooting:")
    print("   1. Ensure you have 'Contributor' role on the resource group")
    print("   2. Verify AZURE_SUBSCRIPTION_ID is correct")
    print("   3. Check that the resource group exists")
    print("   4. Install required package: pip install azure-mgmt-applicationinsights")

## Connect the created application Insights with your foundry project

This is a **portal step** (one-time setup per Foundry resource). Creating an Application Insights resource in Azure does **not** automatically connect it to your Foundry project.

### Steps (MS Foundry portal)
![Connect Application Insights](../images/connect_application_insights.jpg)
1. Open **Foundry portal**: https://ai.azure.com and navigate to your **project**.
2. In the top navigation, select **Operate > Admin > your project name**.

3. select **Add connection**
   - choose **Application Insights** and click continue, set the name you already created in the previous cell , then click **Connect**.
4. After it’s connected, return to **Tracing** to view trace runs and spans.

### Permissions (common gotchas)
- To connect an existing Application Insights resource, you typically need **Contributor** access to the Foundry resource (or Hub).
- To create a new Application Insights resource, you also need **Contributor** on the target resource group.
- To view logs, make sure you have **Log Analytics Reader** on the Application Insights / workspace.

### What you do after connecting
- In code, you can fetch the project’s connection string via `client.telemetry.get_application_insights_connection_string()` and pass it to `configure_azure_monitor(...)` so traces are exported to the linked resource.

In [ ]:
print("\n🔗 Checking Foundry Project connection (AppInsights)...")

try:
    project_cs = client.telemetry.get_application_insights_connection_string()
    os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"] = project_cs
    print("✅ Foundry project has an App Insights connection (Assets > Connections).")
    print(f"   Using project connection string: {project_cs[:60]}...")

except ResourceNotFoundError:
    print("⚠️ No App Insights connection registered in this Foundry project yet.")
    print("   - You created the Azure App Insights resource, but the project connection is not set.")
    print("   - Enable it in Foundry Portal: Operate > Monitoring (or Build > Assets > Connections).")
    print(f"   - You can register this connection string: {connection_string[:60]}...")

In [ ]:
# uuid with YYYYMMDDhhmmss
uuid_suffix = datetime.now().strftime("%Y%m%d%H%M%S")
print(f"uuid_suffix = {uuid_suffix}")

In [ ]:
# Method 2) Create a tracing-enabled agent + generate a simple response (Agent + Tool + Conversations/Responses)


agent_name = f"tracing-agent-{uuid_suffix}"  
instructions = "You are a helpful assistant that answers general questions."

if not AZURE_AI_MODEL_DEPLOYMENT_NAME:
    raise ValueError("AZURE_AI_MODEL_DEPLOYMENT_NAME is empty. Set it in the previous cell.")

with (
    DefaultAzureCredential() as credential,
    AIProjectClient(endpoint=AZURE_AI_PROJECT_ENDPOINT, credential=credential) as project_client,
    project_client.get_openai_client() as openai_client,
 ):
    agent = None
    conversation = None
    try:
        tool = CodeInterpreterTool(container=CodeInterpreterToolAuto())

        agent = project_client.agents.create_version(
            agent_name=agent_name,
            definition=PromptAgentDefinition(
                model=AZURE_AI_MODEL_DEPLOYMENT_NAME,
                instructions=instructions,
                tools=[tool],
            )
        )
        
        print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")
    
        prompt = "\n".join(
            [
                "Use the code interpreter tool to run Python and compute:",
                "",
                "1) Factorial of 100",
                "2) The first 15 Fibonacci numbers (starting from 0, 1)",
                "3) All prime numbers less than 100",
                "",
                "Return only the results in this format:",
                "- factorial_100: <integer>",
                "- fibonacci_15: [..15 ints..]",
                "- primes_lt_100: [..]",
            ]
        )
                
        conversation = openai_client.conversations.create(
            items=[
                {
                    "type": "message",
                    "role": "user",
                    "content": prompt,
                }
            ],
        )
        print(f"Created conversation (id: {conversation.id})")
        
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            input="",
        )
        print(f"Response output: {response.output_text}")


    except Exception as e:
        print(f"Error during agent creation: {repr(e)}")
        


## Check the tracing of agents


### Steps (MS Foundry portal)
1. Open **Foundry portal**: https://ai.azure.com and navigate to your **project**.
2. In the top navigation, select **Build > Agents > your agent name**.
![Agent Tracing](../images/agent_tracing.png)
3. select **Trace ID** to view traces and spans. 
![Agent Tracing Detail](../images/agent_tracing_detail.png)

# Case 3: Assets overview

In [ ]:
# Assets overview: list agents (agent list)
print("🚀 Assets overview - Agents")
print("=" * 80)
def _safe_getattr(obj, name, default=None):
    return getattr(obj, name, default)

try:
    agents = list(client.agents.list())

    if not agents:
        print("⚠️ No deployed agents found.")
    else:
        print(f"📊 {len(agents)} agent(s) found:\n")
        for agent in agents:
            print(f"agent name:{agent.name}")
            print(f"agent version: {agent.versions.get('latest').get('version')}")
            print(f"model name: {agent.versions.get('latest').get('definition').get('model')}")

            # “health”를 SDK가 직접 주지 않으므로, metadata에 있으면 표시(없으면 None)
            metadata = _safe_getattr(agent.versions.get('latest'), "metadata", {}) or {}
            static_health_hint = metadata.get("health") or metadata.get("status")

            print(static_health_hint)

except Exception as e:
    print(f"⚠️ Failed to list agents: {e}")

In [ ]:
# Assets overview: models Overview
print("🚀 Assets overview - Models")
print("=" * 80)

try:
    models = list(client.deployments.list())
    if not models:
        print("⚠️ No deployed models found.")
    else:
        print(f"📊 {len(models)} model(s) found:\n")
        for model in models:
            print(model)
            print(f"model name:{model.name}")
            print(f"model version: {model.get('modelVersion')}")
            print(f"model sku: {model.get('sku')}")

except Exception as e:
    print(f"⚠️ Failed to list models: {e}")

In [ ]:
# Assets overview: Connections Overview
print("🚀 Assets overview - Connections")
print("=" * 80)

try:
    connections = list(client.connections.list())
    if not connections:
        print("⚠️ No deployed connections found.")
    else:
        print(f"📊 {len(connections)} connection(s) found:\n")
        for connection in connections:
            print(f"connection name:{connection.name}")
            print(f"connection type: {connection.get('type')}")
            print(f"connection target: {connection.get('target')}")
except Exception as e:
    print(f"⚠️ Failed to list connections: {e}")

# Case 4: Security overview

# Case 5: Governance overview

## Additional resources

- [Azure AI Foundry Control Plane overview](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/overview?view=foundry)
- [Monitoring across fleet](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/monitoring-across-fleet?view=foundry)
- [Register a custom agent](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/register-custom-agent?view=foundry)
- [Quickstart: create a guardrail policy](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/quickstart-create-guardrail-policy?view=foundry)
- [Manage compliance and security](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-manage-compliance-security?view=foundry)
- [Optimize cost and performance](https://learn.microsoft.com/en-us/azure/ai-foundry/control-plane/how-to-optimize-cost-performance?view=foundry)

# Wrap-up

In this notebook, you did a quick Control Plane-style check using the SDK.

- Good fit for SDK: listing agents/connections, scripting inventory checks
- Portal-first tasks: quota changes, compliance/security dashboards, admin controls

Suggested next steps:
1. Review Fleet/Quota dashboards in the Portal
2. Automate periodic inventory checks in CI
3. Configure governance controls (guardrails/compliance) for production